In [1]:
# Cell 1 — Install & verify required packages in the active Jupyter kernel
%pip install --upgrade yfinance pandas openpyxl requests tqdm python-dateutil

# verify
import importlib
for pkg in ["yfinance", "pandas", "openpyxl", "requests", "tqdm", "dateutil"]:
    try:
        importlib.import_module(pkg)
        print(f"✅ {pkg} installed")
    except Exception as e:
        print(f"❌ {pkg} missing: {e}")


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 7.7 MB/s eta 0:00:02
   ----- ---------------------------------- 1.6/11.0 MB 4.1 MB/s eta 0:00:03
   --------- ------------------------------ 2.6/11.0 MB 4.7 MB/s eta 0:00:02
   -------------- ------------------------- 3.9/11.0 MB 5.0 MB/s eta 0:00:02
   ------------------ --------------------- 5.0/11.0 MB 5.0 MB/s eta 0:00:02
   ---------------------- ----------------- 6.3/11.0 MB 5.2 MB/s eta 0:00:01
   --------------------------- ------------ 7.6/11.0 MB 5.4 MB/s eta 0:00:01
   ------------------------------- -------- 8.7/11.0 MB 5.4 MB/s eta 0:00:01
   ----------------------------------- ---- 9.7/11.0 MB 5.4 MB/s eta 0:00:01
   ---------------------------------------  10.7/11.0 MB 5.4 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 5.3 MB/s eta 0:00:00

  

In [2]:
# Cell 2 — Configuration (edit only these paths if needed)
import os, sys, time, math, json
from pathlib import Path

# === Edit these paths to match your environment (use raw string or forward slashes) ===
INPUT_FILE = r"C:\Users\KIIT\Downloads\stock_list.txt"
OUT_BALANCE_XLSX = r"C:\Users\KIIT\Downloads\all_companies_balancesheet.xlsx"
OUT_OVERVIEW_XLSX = r"C:\Users\KIIT\Downloads\all_companies_overview.xlsx"
ERRORS_CSV = r"C:\Users\KIIT\Downloads\errors.csv"
MANUAL_MAP_CSV = r"C:\Users\KIIT\Downloads\manual_mapping_suggestions.csv"

# Processing config
THREADS = 6                # concurrency; decrease if you get throttled
REQUEST_RETRIES = 3        # attempts for network calls
BASE_SLEEP = 0.6           # base sleep between calls (seconds)
TIMEOUT = 15               # HTTP timeout
# Overview column order to match your earlier file
OVERVIEW_COLUMNS = ["Company", "Yahoo_Ticker", "Price", "Market_Cap", "Revenue", "Net_Profit", "PE_Ratio", "ROE", "Sector", "Industry"]
# don't change below
INPUT_FILE = str(Path(INPUT_FILE))
print("Config set. Input:", INPUT_FILE)


Config set. Input: C:\Users\KIIT\Downloads\stock_list.txt


In [3]:
# Cell 3 — Imports & helper utilities
import requests
import pandas as pd
import yfinance as yf
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm
import re
from urllib.parse import quote_plus

# small helper: safe Excel sheet name
def safe_sheet_name(name: str) -> str:
    # remove forbidden characters and truncate to 31 chars
    name = re.sub(r'[:\\/?*\[\]]', '_', str(name))
    return name[:31]

# normalize raw ticker lines
def normalize_raw_line(line: str) -> str:
    if not line:
        return ""
    s = line.strip()
    # If file contains comma-separated or extra info, keep the first token
    s = s.split()[0]
    # remove common suffixes like _full
    if "_" in s:
        s = s.split("_")[0]
    return s.strip()

# HTTP wrapper with retries
def safe_get(url, headers=None, params=None, timeout=TIMEOUT, tries=REQUEST_RETRIES):
    last_exc = None
    for attempt in range(tries):
        try:
            r = requests.get(url, headers=headers, params=params, timeout=timeout)
            r.raise_for_status()
            return r
        except Exception as e:
            last_exc = e
            time.sleep(BASE_SLEEP * (2 ** attempt))
    raise last_exc


In [4]:
# Cell 4 — Yahoo search fallback function (find possible symbol matches)
def yahoo_search(query: str, limit:int=8):
    """
    Uses Yahoo's public search endpoint to find matching symbols.
    Returns a list of dicts containing keys like 'symbol','shortname','exch','type'
    """
    url = "https://query1.finance.yahoo.com/v1/finance/search"
    params = {"q": query, "quotesCount": limit, "newsCount": 0}
    try:
        r = safe_get(url, params=params)
        j = r.json()
        results = j.get("quotes", []) + j.get("news", [])
        candidates = []
        for item in j.get("quotes", []):
            # keep only equity / funds; we will filter later for .NS or NSE
            sym = item.get("symbol")
            name = item.get("shortname") or item.get("longname") or item.get("quoteType")
            exch = item.get("exchange")
            candidates.append({"symbol": sym, "name": name, "exchange": exch, "raw": item})
        return candidates
    except Exception:
        return []


In [5]:
# Cell 5 — Main fetch function per ticker (tries multiple resolution strategies)
def resolve_and_fetch(raw_ticker: str):
    """
    Attempt to get a working Yahoo ticker for an NSE stock and fetch its balance sheet and overview.
    Returns:
      - success (bool)
      - yahoo_symbol (str or None)
      - company_name (str or None)
      - balance_df (pd.DataFrame)  -> annual balance sheet transposed (Periods as rows)
      - overview_row (dict) matching OVERVIEW_COLUMNS
      - error_message (str or None)
    """
    rt = normalize_raw_line(raw_ticker)
    if not rt:
        return False, None, None, pd.DataFrame(), {}, "empty input"

    tried_symbols = []
    errors = []

    # candidate generation rules:
    candidates_to_try = []

    # 1) if looks like a Yahoo symbol already (contains .)
    if "." in rt:
        candidates_to_try.append(rt)
    # 2) direct with .NS suffix
    candidates_to_try.append(rt + ".NS")
    # 3) uppercase without .NS then with .NS
    t_up = re.sub(r'[^A-Za-z0-9&]', '', rt).upper()
    candidates_to_try.append(t_up + ".NS")
    # 4) if & in ticker or official name variants: try replacing & with AND and vice versa
    if "&" in t_up:
        candidates_to_try.append(t_up.replace("&","AND") + ".NS")
    if "AND" in t_up:
        candidates_to_try.append(t_up.replace("AND","&") + ".NS")
    # dedupe preserve order
    seen = set()
    candidates_to_try = [c for c in candidates_to_try if not (c in seen or seen.add(c))]

    # helper to attempt a single yahoo symbol
    def try_symbol(sym):
        try:
            tried_symbols.append(sym)
            t = yf.Ticker(sym)
            # attempt to fetch info (this triggers a network call inside yfinance)
            info = {}
            try:
                info = t.info or {}
            except Exception as e:
                # yfinance 404 or JSON parse issues
                info = {}
            # if info contains symbol or longName, treat as found
            if info and (info.get("symbol") or info.get("longName") or info.get("shortName")):
                # fetch balance sheet; prefer annual balance sheet
                try:
                    bal = t.balance_sheet
                    # yfinance returns DataFrame with columns = periods; convert to rows per period
                    if bal is not None and not bal.empty:
                        bal_df = bal.transpose().reset_index().rename(columns={"index":"Unnamed: 0"})
                    else:
                        bal_df = pd.DataFrame()
                    # construct overview row
                    row = {
                        "Company": info.get("longName") or info.get("shortName") or sym,
                        "Yahoo_Ticker": sym,
                        "Price": info.get("currentPrice") or info.get("previousClose"),
                        "Market_Cap": info.get("marketCap"),
                        "Revenue": info.get("totalRevenue") or info.get("trailingAnnualRevenue"),
                        "Net_Profit": info.get("netIncomeToCommon") or info.get("netIncome"),
                        "PE_Ratio": info.get("trailingPE") or info.get("forwardPE"),
                        "ROE": (info.get("returnOnEquity") * 100) if info.get("returnOnEquity") is not None and abs(info.get("returnOnEquity")) <=1 else info.get("returnOnEquity"),
                        "Sector": info.get("sector"),
                        "Industry": info.get("industry")
                    }
                    # ensure order
                    overview_row = {k: row.get(k) for k in OVERVIEW_COLUMNS}
                    return True, sym, row["Company"], bal_df, overview_row, None
                except Exception as e:
                    return False, None, None, pd.DataFrame(), {}, f"yfinance fetch error: {e}"
            else:
                return False, None, None, pd.DataFrame(), {}, f"yfinance returned no info for {sym}"
        except Exception as e:
            return False, None, None, pd.DataFrame(), {}, f"exception trying symbol {sym}: {e}"

    # 1. Try candidate symbols deterministically
    for sym in candidates_to_try:
        ok, sym_found, comp_name, bal_df, overview_row, err = try_symbol(sym)
        if ok:
            return True, sym_found, comp_name, bal_df, overview_row, None
        else:
            errors.append((sym, err))

    # 2. If deterministic tries failed, attempt a Yahoo Search by using the raw token or a cleaned version
    search_queries = [rt, t_up, re.sub(r'\d+$','', t_up)]  # try raw, uppercase, remove trailing digits
    for q in search_queries:
        if not q:
            continue
        candidates = yahoo_search(q)
        # prefer NSE / .NS candidates if present in results
        # candidates are dicts: {'symbol':..., 'name':..., 'exchange':...}
        prioritized = []
        for c in candidates:
            sym = c.get("symbol")
            if not sym:
                continue
            # prefer symbols that end with .NS or mention NSE / India in exchange
            if sym.endswith(".NS") or (c.get("exchange") and "NSE" in str(c.get("exchange")).upper()):
                prioritized.insert(0, sym)  # high priority
            else:
                prioritized.append(sym)
        # try found prioritized symbols
        for sym in prioritized:
            ok, sym_found, comp_name, bal_df, overview_row, err = try_symbol(sym)
            if ok:
                return True, sym_found, comp_name, bal_df, overview_row, None
            else:
                errors.append((sym, err))

    # 3. If still not found, return failure with suggestions (candidates tried and last errors)
    suggestion_symbols = [c for c,_ in errors][:10]
    msg = f"Not resolved. Tried: {tried_symbols[:10]}. Suggestions from search: {suggestion_symbols}."
    return False, None, None, pd.DataFrame(), {}, msg


In [6]:
# Cell 6 — Read input tickers and run batch with concurrency; store intermediate results in memory
# NOTE: you can run this cell and it will process all tickers. For large lists it takes time.
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    raw_lines = [ln.strip() for ln in f if ln.strip()]

print(f"Loaded {len(raw_lines)} tickers to process.")

successful = []
overview_rows = []
balances_map = {}   # key -> (symbol, company_name, dataframe)
errors = []

# We'll use a thread pool and tqdm progress bar
with ThreadPoolExecutor(max_workers=THREADS) as ex:
    future_to_raw = {ex.submit(resolve_and_fetch, rl): rl for rl in raw_lines}
    for fut in tqdm(as_completed(future_to_raw), total=len(future_to_raw), desc="Overall progress"):
        raw = future_to_raw[fut]
        try:
            ok, symbol, company_name, bal_df, overview_row, err = fut.result()
            if ok:
                successful.append((raw, symbol))
                # ensure sheet-safe company label
                sheet_label = f"{symbol}__{company_name}" if company_name else symbol
                sheet_label = safe_sheet_name(sheet_label)
                balances_map[sheet_label] = (symbol, company_name, bal_df)
                overview_rows.append(overview_row)
                print(f"[OK] {raw} -> {symbol} ({company_name})")
            else:
                errors.append((raw, err))
                print(f"[FAIL] {raw} -> {err}")
        except Exception as e:
            errors.append((raw, str(e)))
            print(f"[EXC] {raw} -> {e}")

print(f"Processing finished. Success: {len(successful)}, Failures: {len(errors)}")


Loaded 2135 tickers to process.


Overall progress:   0%|          | 0/2135 [00:00<?, ?it/s]

[OK] 20MICRONS_full -> 20MICRONS.NS (20 Microns Limited)
[OK] 3MINDIA_full -> 3MINDIA.NS (3M India Limited)
[OK] 21STCENMGM_full -> 21STCENMGM.NS (Twentyfirst Century Management Services Limited)
[OK] 3PLAND_full -> 3PLAND.NS (3P Land Holdings Ltd)
[OK] 5PAISA_full -> 5PAISA.NS (5paisa Capital Limited)
[OK] 360ONE_full -> 360ONE.NS (360 One Wam Limited)
[OK] 63MOONS_full -> 63MOONS.NS (63 moons technologies limited)
[OK] AADHARHFC_full -> AADHARHFC.NS (AADHAR HOUSING FINANCE L)
[OK] AAKASH_full -> AAKASH.NS (Aakash Exploration Services Limited)
[OK] A2ZINFRA_full -> A2ZINFRA.NS (A2Z Infra Engineering Limited)
[OK] AAATECH_full -> AAATECH.NS (AAA Technologies Limited)
[OK] AARON_full -> AARON.NS (Aaron Industries Limited)
[OK] AAREYDRUGS_full -> AAREYDRUGS.NS (Aarey Drugs & Pharmaceuticals Limited)
[OK] AARTIDRUGS_full -> AARTIDRUGS.NS (Aarti Drugs Limited)
[OK] AARTIIND_full -> AARTIIND.NS (Aarti Industries Limited)
[OK] AARTIPHARM_full -> AARTIPHARM.NS (Aarti Pharmalabs Limited)
[OK] 

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: AREM.NS"}}}


[OK] ARIES_full -> ARIES.NS (Aries Agro Limited)
[OK] ARCHIES_full -> ARCHIES.NS (Archies Limited)
[OK] ARFIN_full -> ARFIN.NS (Arfin India Limited)
[OK] ARIHANTSUP_full -> ARIHANTSUP.NS (Arihant Superstructures Limited)
[OK] ARISINFRA_full -> ARISINFRA.NS (ARISINFRA SOLUTIONS LTD)
[OK] ARKADE_full -> ARKADE.NS (ARKADE DEVELOPERS LIMITED)
[OK] ARMANFIN_full -> ARMANFIN.NS (Arman Financial Services Limited)
[OK] ARROWGREEN_full -> ARROWGREEN.NS (Arrow Greentech Limited)
[OK] ARSSINFRA_full -> ARSSINFRA.NS (ARSS Infrastructure Projects Limited)
[OK] AROGRANITE_full -> AROGRANITE.NS (Aro Granite Industries Limited)
[OK] ARVEE_full -> ARVEE.NS (Arvee Laboratories (India) Ltd.)
[OK] ARTNIRMAN_full -> ARTNIRMAN.NS (Art Nirman Limited)
[OK] ARTEMISMED_full -> ARTEMISMED.NS (Artemis Medicare Services Limited)
[OK] ARVSMART_full -> ARVSMART.NS (Arvind SmartSpaces Limited)
[OK] ARVIND_full -> ARVIND.NS (Arvind Limited)
[OK] ASAHIINDIA_full -> ASAHIINDIA.NS (Asahi India Glass Limited)
[OK] ASAHIS

...

[OK] MMP_full -> MMP.NS (MMP Industries Limited)
[OK] MOBIKWIK_full -> MOBIKWIK.NS (ONE MOBIKWIK SYSTEMS LTD)
[OK] MMTC_full -> MMTC.NS (MMTC Limited)
[OK] MODIRUBBER_full -> MODIRUBBER.NS (Modi Rubber Limited)
[OK] MODTHREAD_full -> MODTHREAD.NS (Modern Threads (India) Limited)
[OK] MODISONLTD_full -> MODISONLTD.NS (Modison Limited)
[OK] MOHITIND_full -> MOHITIND.NS (Mohit Industries Limited)
[OK] MOIL_full -> MOIL.NS (MOIL Limited)
[OK] MOLDTKPAC_full -> MOLDTKPAC.NS (Mold-Tek Packaging Limited)
[OK] MOKSH_full -> MOKSH.NS (Moksh Ornaments Limited)
[OK] MOLDTECH_full -> MOLDTECH.NS (Mold-Tek Technologies Limited)
[OK] MOL_full -> MOL.NS (Meghmani Organics Limited)
[OK] MONTECARLO_full -> MONTECARLO.NS (Monte Carlo Fashions Limited)
[OK] MORARJEE_full -> MORARJEE.NS (Morarjee Textiles Limited)
[OK] MONARCH_full -> MONARCH.NS (Monarch Networth Capital Limited)
[OK] MOREPENLAB_full -> MOREPENLAB.NS (Morepen Laboratories Limited)
[OK] MOTHERSON_full -> MOTHERSON.NS (Samvardhana Motherson

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: SSPOWER.NS"}}}


[OK] SRHHYPOLTD_full -> SRHHYPOLTD.NS (Sree Rayalaseema Hi-Strength Hypo Limited)
[OK] SRF_full -> SRF.NS (SRF Limited)
[OK] SRM_full -> SRM.NS (SRM CONTRACTORS LIMITED)
[OK] SSWL_full -> SSWL.NS (Steel Strips Wheels Limited)
[OK] STALLION_full -> STALLION.NS (STALLION IND FLUOROCHEM L)
[OK] SSDL_full -> SSDL.NS (SARASWATI SAREE DEPOT LTD)
[OK] STANLEY_full -> STANLEY.NS (STANLEY LIFESTYLES LTD)
[OK] STARHEALTH_full -> STARHEALTH.NS (Star Health and Allied Insurance Company Limited)
[OK] STARCEMENT_full -> STARCEMENT.NS (Star Cement Limited)
[OK] STCINDIA_full -> STCINDIA.NS (The State Trading Corporation of India Limited)
[OK] STAR_full -> STAR.NS (Strides Pharma Science Limited)
[OK] STARTECK_full -> STARTECK.NS (Starteck Finance Limited)
[OK] STARPAPER_full -> STARPAPER.NS (Star Paper Mills Limited)
[OK] STEELCITY_full -> STEELCITY.NS (Steel City Securities Limited)
[OK] STEELCAS_full -> STEELCAS.NS (Steelcast Limited)
[OK] STEELXIND_full -> STEELXIND.NS (Steel Exchange India Limite

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: SURANATP.NS"}}}


[OK] SURANASOL_full -> SURANASOL.NS (Surana Solar Limited)
[OK] SURAJLTD_full -> SURAJLTD.NS (Suraj Limited)
[OK] SURYALAXMI_full -> SURYALAXMI.NS (Suryalakshmi Cotton Mills Limited)
[OK] SURYAROSNI_full -> SURYAROSNI.NS (Surya Roshni Limited)
[OK] SUTLEJTEX_full -> SUTLEJTEX.NS (Sutlej Textiles and Industries Limited)
[OK] SURYODAY_full -> SURYODAY.NS (Suryoday Small Finance Bank Limited)
[OK] SURAKSHA_full -> SURAKSHA.NS (SURAKSHA DIAGNOSTIC LTD)
[OK] SUVIDHAA_full -> SUVIDHAA.NS (Suvidhaa Infoserve Limited)
[OK] SUZLON_full -> SUZLON.NS (Suzlon Energy Limited)
[OK] SUVEN_full -> SUVEN.NS (Suven Life Sciences Limited)
[OK] SUYOG_full -> SUYOG.NS (Suyog Telematics Limited)
[OK] SVLL_full -> SVLL.NS (Shree Vasu Logistics Limited)
[OK] SWANENERGY_full -> SWANENERGY.NS (Swan Corp Limited)
[OK] SVPGLOB_full -> SVPGLOB.NS (SVP Global Textiles Limited)
[OK] SWARAJENG_full -> SWARAJENG.NS (Swaraj Engines Limited)
[OK] SWIGGY_full -> SWIGGY.NS (SWIGGY LIMITED)
[OK] SWELECTES_full -> SWELECTES

In [7]:
# Cell 7 — Save Balance sheets workbook (one sheet per company)
print("Saving balances workbook:", OUT_BALANCE_XLSX)
with pd.ExcelWriter(OUT_BALANCE_XLSX, engine="openpyxl") as writer:
    for sheet_label, (symbol, company_name, bal_df) in balances_map.items():
        if bal_df is None or bal_df.empty:
            # put a small note sheet
            pd.DataFrame({"Note":[f"No balance sheet scraped for {symbol} ({company_name})"]}).to_excel(writer, sheet_name=sheet_label[:31], index=False)
        else:
            # ensure Unnamed: 0 exists for years/periods (match your old file layout)
            if "Unnamed: 0" not in bal_df.columns and bal_df.shape[1] > 0:
                # try to rename first col if it's index-like
                bal_df = bal_df.reset_index().rename(columns={"index":"Unnamed: 0"})
            bal_df.to_excel(writer, sheet_name=sheet_label[:31], index=False)
print("Balances saved.")


Saving balances workbook: C:\Users\KIIT\Downloads\all_companies_balancesheet.xlsx
Balances saved.


In [8]:
# Cell 8 — Save Overview (single sheet)
print("Saving overview workbook:", OUT_OVERVIEW_XLSX)
df_overview = pd.DataFrame(overview_rows)
# Ensure all expected columns exist
for c in OVERVIEW_COLUMNS:
    if c not in df_overview.columns:
        df_overview[c] = None
df_overview = df_overview.reindex(columns=OVERVIEW_COLUMNS)
df_overview.to_excel(OUT_OVERVIEW_XLSX, index=False)
print("Overview saved.")


Saving overview workbook: C:\Users\KIIT\Downloads\all_companies_overview.xlsx
Overview saved.


In [9]:
# Cell 9 — Save errors & manual mapping suggestions
print("Saving errors to:", ERRORS_CSV)
pd.DataFrame(errors, columns=["raw_input", "error"]).to_csv(ERRORS_CSV, index=False)

# For manual mapping suggestions, attempt quick suggestions using yahoo_search
manual_suggestions = []
for raw, _ in errors:
    suggs = yahoo_search(raw)
    # flatten top 5 symbols
    top_syms = [s.get("symbol") for s in suggs[:5]]
    manual_suggestions.append({"raw_input": raw, "suggestions": ",".join([str(x) for x in top_syms])})

pd.DataFrame(manual_suggestions).to_csv(MANUAL_MAP_CSV, index=False)
print("Errors and manual mapping suggestions saved.")


Saving errors to: C:\Users\KIIT\Downloads\errors.csv
Errors and manual mapping suggestions saved.
